In [ ]:
import classifier_wlan_utils as utils
import importlib
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
importlib.reload(utils)

In [ ]:
seed = 42
task = "frames"
label = utils.labels[task]
num_classes = utils.classes[task]
print("Label id: ", label)
print("Num classes in that label: ", num_classes)
num_iq_seq = 10000# 500us at 20MHz BW
padding = 'post'
dataset_folder = '../../dataset/waveforms/'
load_model=True
load_dir='../../models/'
load_file = '22042020175144_saved_wavenet_classifier'

In [ ]:
dataset_filename = 'waveforms_2G_n_multi_mobile_app_multi_channel_100k_balanced.mat'
X, Y, Y_pd = utils.get_xy(dataset_folder,dataset_filename, num_iq_seq, padding, num_classes, label)
X_train, X_val, X_test, Y_train, Y_val, Y_test = utils.get_xy_4_training(X,Y,seed)
print(X_train.shape,X_val.shape, X_test.shape)
print(Y_train.shape,Y_val.shape, Y_test.shape)

In [ ]:
dataset_filename_qos = 'waveforms_2G_n_multi_mobile_app_2_only_qos.mat'
X_qos, Y_qos, Y_pd_qos = utils.get_xy(dataset_folder,dataset_filename_qos, num_iq_seq, padding, num_classes, label)
print(X_qos.shape, Y_qos.shape)

In [ ]:
dataset_filename_test2 = 'waveforms_2G_n_multi_mobile_app_multi_channel_21042020_balanced.mat'
X_test2, Y_pd_test2 = utils.get_xy(dataset_folder,dataset_filename_test2, num_iq_seq, padding, num_classes, label)
print(X_test2.shape, Y_test2.shape)

In [ ]:
if load_model:
    load_dir='../../models/'
    load_file = '22042020175144_saved_wavenet_classifier'
    wnc = utils.WaveNetClassifier((2,num_iq_seq), (num_classes,), kernel_size = 2, dilation_depth = 6, n_filters = 32, task = 'classification', regression_range = None, load = load_model, load_dir = load_dir, load_file= load_file)
else:
    wnc = utils.WaveNetClassifier((2,num_iq_seq), (num_classes,), kernel_size = 2, dilation_depth = 6, n_filters = 32)
    wnc.fit(X_train, Y_train, validation_data = (X_val, Y_val),save=True, epochs = 50)

In [ ]:
test_results = wnc.evaluate(X_test, Y_test)
print(test_results)
Y_pred=np.argmax(wnc.predict(X_test),1)
Y_true=np.argmax(Y_test,1)
utils.plot_confusion_matrix_mc(Y_true, Y_pred, ["Mgmt","Ctrl","Data"], "conf_matrix_test.pdf")
print(confusion_matrix(Y_true, Y_pred))
print(confusion_matrix(Y_true, Y_pred, normalize='all'))

In [ ]:
qos_results = wnc.evaluate(X_qos, Y_qos)
print(qos_results)
Y_pred_qos=np.argmax(wnc.predict(X_qos),1)
Y_true_qos=np.argmax(Y_qos,1)
#accuracy_score(Y_pred_qos, Y_true_qos)
utils.plot_confusion_matrix_mc(Y_pred_qos, Y_true_qos, ["Mgmt","Ctrl","Data"], "conf_matrix_qos.pdf")

In [ ]:
test2_results = wnc.evaluate(X_test2, Y_test2)
print(test2_results)
Y_pred_test2=np.argmax(wnc.predict(X_test2),1)
Y_true_test2=np.argmax(Y_test2,1)
accuracy_score(Y_pred_test2, Y_true_test2)
utils.plot_confusion_matrix_mc(Y_pred_test2, Y_true_test2, ["Mgmt","Ctrl","Data"], "conf_matrix_test2.pdf")

In [ ]:
dataset_filename_apps = 'waveforms_2G_n_6_mobile_app_24042020.mat'
X_apps, Y_apps, Y_pd_apps = utils.get_xy(dataset_folder,dataset_filename_apps, num_iq_seq, padding, label)

In [ ]:
test_results_apps = wcn_model.evaluate(X_apps, Y_apps)
print(test_results_apps)
Y_pred_apps=np.argmax(wnc.predict(X_apps),1)
Y_true_apps=np.argmax(Y_apps,1)
plot_confusion_matrix_mc(Y_pred_apps, Y_true_apps, ["Mgmt","Ctrl","Data"], "conf_matrix_apps.pdf")

In [ ]:
#from sklearn.metrics import average_precision_score
#average_precision = average_precision_score(Y_test, wcn_model.predict(X_test))
#print('Average precision-recall score: {0:0.2f}'.format(
#      average_precision))
#print(confusion_matrix(Y_true_apps, Y_pred_apps))
#print(confusion_matrix(Y_true_apps, Y_pred_apps, normalize='all'))
#print(cm)